In [30]:
run_name = ""
result_path = "../Training/results"
training_step_length = 10_000

In [2]:
import glob
import json
import matplotlib.pyplot as plt

# Evaluation Plot

The following cell loads all `evaluation<...>.json` files and plots their values.

In [ ]:
evallist = glob.glob(f"{result_path}/{run_name}*/evaluation*.json")

plotdict = {}
for eval in evallist:
    with open(eval) as e:
        jdict = json.load(e)
        trackName = jdict["trackName"]
        if trackName not in plotdict:
            plotdict[trackName] = list()
        plotdict[trackName].append(jdict["result"])

for track in plotdict.keys():
    plt.plot(range(training_step_length, (len(plotdict[track]) + 1) * training_step_length, training_step_length), plotdict[track], label=track)
plt.title(run_name)
plt.legend()
plt.show()